In [1]:
import getpass
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
input_msg = input("請輸入問題")
result = model.invoke(
    [
        SystemMessage(content="請使用繁體中文回答。"),
        HumanMessage(content=input_msg),
    ]
)
display(Markdown(result.content))



c:\Users\User\anaconda3\envs\L-AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# Useful Tools
def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))


In [4]:
## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser() # stop the output

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print(zsc_call("Should I take the next exit, or keep going to the next one?"))

car


In [5]:
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State:  {'topic': 'boat'}


'The old sailboat, its weathered hull whispering tales of distant shores, dreamt of escaping its moorings and dancing with the wind once more. \n'

In [ ]:
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})
pprint("Final Output: ", output)

In [1]:
import getpass
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import gradio as gr

from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough, RunnableMap, RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain.output_parsers import PydanticOutputParser

from functools import partial
from typing import List, Union, Optional, Dict, Iterable
from operator import itemgetter
from pydantic import BaseModel, Field

from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# Useful Tools
def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def RExtract(pydantic_class, llm, prompt):
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

# Database
class PersonalInfoBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    name: str = Field('unknown', description="Chatting user's name, unknown if unknown")
    height: float = Field(0, description="Chatting user's height in centimeter, 0 if unknown")
    weight: float = Field(0, description="Chatting user's weight in kilogram, 0 if unknown")
    gender: str = Field('unknown', description="Chatting user's weight, unknown if unknown")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")
    summary: str = Field("", description="Summary of discussion so far")
    
    
# prompt
main_bot_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "你的身分是一位專業資深有禮貌的的中醫師，名字叫做L AI"
        "主要使用繁體中文回答"
        "負責分析使用者症狀並讓使用者做諮詢"
        "只會回答中醫相關的問題"
        "當使用者想要諮詢時，詢問其個人資訊，根據: {info_base}"
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    " Record your response in the 'response' tag to continue the conversation."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {info_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)


# LLM Model & Chain
load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
instruct_llm = model | StrOutputParser()
chat_llm = model | StrOutputParser()
external_chain = main_bot_prompt | chat_llm


knowbase_getter = lambda x: PersonalInfoBase()
knowbase_getter = RExtract(PersonalInfoBase, instruct_llm, parser_prompt)


## These components integrate to make your internal chain
internal_chain = (
    RunnableAssign({'info_base' : knowbase_getter})
)

state = {'info_base' : PersonalInfoBase()}

# Chat implementation
def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)

    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer if return_buffer else token

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=8):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

# history is of format [[User response 0, Bot response 0], ...]
chat_history = [[None, "您好，我是AI中醫師L AI，請問有什麼需要協助的嗎？"]]

# Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = chat_gen,
    history = chat_history
)

# chatbot = gr.Chatbot(value=[[None, "Hello! I'm your SkyFlow agent! How can I help you?"]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue().launch(debug=True, share=True)

<>:51: SyntaxWarning: invalid escape sequence '\]'
<>:52: SyntaxWarning: invalid escape sequence '\['
<>:51: SyntaxWarning: invalid escape sequence '\]'
<>:52: SyntaxWarning: invalid escape sequence '\['
C:\Users\Eric Potter\AppData\Local\Temp\ipykernel_26696\834572703.py:51: SyntaxWarning: invalid escape sequence '\]'
  .replace("\]", "]")
C:\Users\Eric Potter\AppData\Local\Temp\ipykernel_26696\834572703.py:52: SyntaxWarning: invalid escape sequence '\['
  .replace("\[", "[")



[ Agent ]: 您好，我是AI中醫師L AI，請問有什麼需要協助的嗎？

[ Agent ]: 


<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: inv

好的，請問您方便告知您的姓名、性別、身高、體重以及目前遇到的問題嗎？我會根據您的資訊提供更精準的建議。



[ Agent ]: 


<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: inv

請問您方便告知您的姓名、性別、身高、體重以及目前遇到的問題嗎？我會根據您的資訊提供更精準的建議。





[ Agent ]: 


<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: invalid escape sequence '\['
<unknown>:7: SyntaxWarning: invalid escape sequence '\]'
<unknown>:8: SyntaxWarning: inv

請問您方便告知您的姓名、性別、身高、體重以及目前遇到的問題嗎？我會根據您的資訊提供更精準的建議。


